In [ ]:
!nvidia-smi


Mon May 10 07:17:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


### **Create a new folder yolov3 in your drive**

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/yolov3"

In [ ]:
%cd /content/gdrive/My Drive/yolov3

/content/gdrive/My Drive/yolov3


### **Download face mask dataset from kaggle**

After downloading and unzipping, you would see two folders two folders :-

1.   images
2.   annotations



In [ ]:
!kaggle datasets download -d andrewmvd/face-mask-detection --unzip --force



 97% 387M/398M [00:03<00:00, 138MB/s]
100% 398M/398M [00:03<00:00, 121MB/s]


Create a folder mask_yolo_train where will add our training data

In [ ]:
!mkdir mask_yolo_train
%ls


### **Convert xml files to txt**

The annotations folder contains xml files. xml_to_yolo.py will fetch all the requiered attributes from the xml file and save it as a text file. Make sure to place xml_to_yolo.py in yolov3 folder.

In [ ]:
!python xml_to_yolo.py --input=annotations --output=/content/gdrive/MyDrive/yolov3/mask_yolo_train/

### **Count the number of files in mask_yolo_train**

In [ ]:
#after converting xml files to txt, the output should be 853
!ls mask_yolo_train// -1 | wc -l


In [ ]:
#total number of images in our dataset
!ls images// -1 | wc -l


853


### **Copy images to mask_yolo_train**

In [ ]:
!cp -r images/. mask_yolo_train


### **Check the count of Images and Text files in mask_yolo_train and mask_yolo_test folders**

First create a mask_yolo_test folder in the yolov3 folder
We need to divide our dataset for training and testing. I moved 88 images and their corresponding txt files to mask_yolo_test folder. So mask_yolo_train would contain 765 images.

In [ ]:
#detect the count of images and text files in mask_yolo_test and mask_yolo_train folder
%cd mask_yolo_train
!ls -F | grep .png | wc -l
!ls -F | grep .txt | wc -l
%cd ..

%cd mask_yolo_test
!ls -F | grep .png | wc -l
!ls -F | grep .txt | wc -l
%cd ..


/content/gdrive/My Drive/yolov3/mask_yolo_train
765
765
/content/gdrive/My Drive/yolov3
/content/gdrive/My Drive/yolov3/mask_yolo_test
88
88
/content/gdrive/My Drive/yolov3


### **Create train.txt and test.txt which will contain the absolute paths**

This code will creat train.txt and test.txt which will include the absolute paths of the training data and testing data respectively.

In [ ]:
import os

for dataset, txtname in zip(['mask_yolo_train/', 'mask_yolo_test/'], ['train.txt', 'test.txt']):
    image_files = []
    # Given we are already being located into /face_mask_detection folder
    os.chdir(os.path.abspath(dataset))
    count=0
    for filename in os.listdir(os.getcwd()):
      if filename.endswith(".png"):
          image_files.append(os.path.abspath(os.getcwd()) + '/' + filename)

    os.chdir("..")
    with open(txtname, "w") as outfile:
        for image in image_files:
            outfile.write(image)
            outfile.write("\n")
        outfile.close()

### **Clone the darkent repository in your drive**

In [ ]:
!git clone https://github.com/AlexeyAB/darknet


Cloning into 'darknet'...
remote: Enumerating objects: 14997, done.
remote: Total 14997 (delta 0), reused 0 (delta 0), pack-reused 14997
Receiving objects: 100% (14997/14997), 13.35 MiB | 8.90 MiB/s, done.
Resolving deltas: 100% (10193/10193), done.
Checking out files: 100% (2024/2024), done.


### **Copy train.txt and test.txt to /darknet/data folder**

In [ ]:
!cp train.txt ../darknet/data
!cp test.txt ../darknet/data



cp: cannot create regular file '../darknet/data': No such file or directory
cp: cannot create regular file '../darknet/data': No such file or directory


In [ ]:
%cd darknet

[Errno 2] No such file or directory: 'darknet'
/content/gdrive/My Drive/yolov3


### **Check if nvidia GPU is enabled**

In [ ]:
!nvidia-smi


Sat May  8 13:55:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#configure
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
#compile
!make 


chmod +x *.sh


In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2021-05-08 15:37:26--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M  40.2MB/s    in 6.1s    

2021-05-08 15:37:32 (38.8 MB/s) - ‘yolov3.weights.1’ saved [248007048/248007048]



In [ ]:
#download weights of the pre-trained model
!wget https://pjreddie.com/media/files/darknet53.conv.74


--2021-05-08 15:37:37--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  41.6MB/s    in 4.1s    

2021-05-08 15:37:42 (37.9 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



In [ ]:
%cd ..
%cd darknet


/content/gdrive/My Drive
/content/gdrive/My Drive/darknet


In [ ]:
!chmod +x ./darknet


## **Train!**

Here we are going to start our training. 
Note that in our darknet/data folder we must have 


*   face_mask.data
*   face_mask.names

And in the darknet/cfg folder :


*   face_mask.cfg

And yolov3 folder must have a backup folder for saving weights.



In [ ]:
#use this command when training for the first time
!./darknet detector train data/face_mask.data cfg/face_mask.cfg darknet53.conv.74 -dont_show


Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.860495), count: 2, class_loss = 0.030032, iou_loss = 0.046764, total_loss = 0.076796 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.822001), count: 1, class_loss = 0.005774, iou_loss = 0.018694, total_loss = 0.024469 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.756316), count: 10, class_loss = 1.777322, iou_loss = 0.511434, total_loss = 2.288756 
 total_bbox = 415017, rewritten_bbox = 0.034215 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.720574), count: 1, class_loss = 0.071342, iou_loss = 0.050123, total_loss = 0.121464 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.880778), count: 2, class_loss = 0.019122, iou_loss = 0.034334, total_loss = 0.053456 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.

In [ ]:
#use this command to load previously saved weigths in the backup folder to continue your training in case it was interrupted before
!./darknet detector train data/face_mask.data cfg/face_mask.cfg backup/face_mask_last.weights -dont_show -i 0 -map -points 0


Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.846071), count: 5, class_loss = 0.701023, iou_loss = 0.156338, total_loss = 0.857361 
 total_bbox = 7223, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.811361), count: 1, class_loss = 0.166228, iou_loss = 0.022898, total_loss = 0.189127 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.842446), count: 13, class_loss = 0.321344, iou_loss = 0.191417, total_loss = 0.512761 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.804620), count: 38, class_loss = 2.618667, iou_loss = 1.431417, total_loss = 4.050084 
 total_bbox = 7275, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.901797), count: 2, class_loss = 0.145561, iou_loss = 0.014175, total_loss = 0.159736 
v3 (mse

## **Test!**

It's time to test our model with the test dataset. Notice how we are using face_mask_best.weights instead of face_mask_last.weigths. This will load the best weights for our model (in my case map@0.5 85.25%)

In [ ]:
!./darknet detector test data/face_mask.data cfg/face_mask.cfg backup/face_mask_best.weights


 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5, 

This function will save the output in a new .jpg file with the bounding boxes and predictions.

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread('/content/gdrive/MyDrive/yolov3/sample/protective-suit-5716753_1280.jpg')
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

imShow('predictions.jpg')

In [ ]:
#use this command to pass a video file. The output is saves in .avi format
!./darknet detector demo data/face_mask.data cfg/face_mask.cfg backup/face_mask_best.weights -dont_show /content/gdrive/MyDrive/yolov3/sample/maskvid.mp4 -i 0 -out_filename result1.avi

Streaming output truncated to the last 5000 lines.
 cvWriteFrame 
Objects:

Bad: 85% 
Good: 100% 
Good: 100% 
Good: 99% 
Good: 96% 

FPS:14.6 	 AVG_FPS:14.3

 cvWriteFrame 
Objects:

Bad: 90% 
Good: 100% 
Good: 100% 
Good: 100% 
Good: 96% 

FPS:14.5 	 AVG_FPS:14.3

 cvWriteFrame 
Objects:

Bad: 73% 
Good: 33% 
Good: 100% 
Good: 100% 
Good: 100% 
Good: 96% 

FPS:14.2 	 AVG_FPS:14.3

 cvWriteFrame 
Objects:

Bad: 46% 
Good: 46% 
Good: 100% 
Good: 100% 
Good: 100% 
Good: 95% 

FPS:14.5 	 AVG_FPS:14.3

 cvWriteFrame 
Objects:

Bad: 34% 
Good: 68% 
Good: 100% 
Good: 100% 
Good: 100% 
Good: 95% 

FPS:14.5 	 AVG_FPS:14.3

 cvWriteFrame 
Objects:

Good: 100% 
Good: 100% 
Good: 100% 
Good: 95% 
Good: 87% 

FPS:14.8 	 AVG_FPS:14.3

 cvWriteFrame 
Objects:

Bad: 33% 
Good: 100% 
Good: 100% 
Good: 100% 
Good: 95% 
Good: 78% 

FPS:14.5 	 AVG_FPS:14.3

 cvWriteFrame 
Objects:

Bad: 48% 
Good: 62% 
Good: 100% 
Good: 100% 
Good: 100% 
Good: 95% 

FPS:14.3 	 AVG_FPS:14.3

 cvWriteFrame 
Objects:

Bad: 